In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_LV
from utils_training import LVTrainingData
from utils_training import evaluate_LVmodel

In [4]:
from behavior_model import LVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 7, 0.25
acc_std_dev = 0.07
Inputs, Outputs, Weights = LVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_05_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_LV(Inputs_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_LV(Inputs_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = LVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_lv_val_05.pth'
best_train_model_path = f'.\\weights\\bc_lv_train_05.pth'
best_trainval_model_path = f'.\\weights\\bc_lv_trainval_05.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_LVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_LVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 387.87,      Loss_min: 381.57, Loss_max: 396.26 
      Loss_mean_val: 387.34, Loss_min_val: 381.85,      Loss_max_val: 395.91 
      Loss_mean_trainval: 387.4
Epoch 2/4000, Loss_mean: 384.56,      Loss_min: 382.8, Loss_max: 386.47 
      Loss_mean_val: 383.52, Loss_min_val: 375.16,      Loss_max_val: 389.99 
      Loss_mean_trainval: 383.63
Epoch 3/4000, Loss_mean: 379.15,      Loss_min: 375.31, Loss_max: 382.61 
      Loss_mean_val: 379.54, Loss_min_val: 367.43,      Loss_max_val: 387.48 
      Loss_mean_trainval: 379.5
Epoch 4/4000, Loss_mean: 373.56,      Loss_min: 367.28, Loss_max: 378.85 
      Loss_mean_val: 375.24, Loss_min_val: 354.69,      Loss_max_val: 380.91 
      Loss_mean_trainval: 375.06
Epoch 5/4000, Loss_mean: 369.48,      Loss_min: 367.13, Loss_max: 371.96 
      Loss_mean_val: 370.64, Loss_min_val: 348.64,      Loss_max_val: 380.98 
      Loss_mean_trainval: 370.51
Epoch 6/4000, Loss_mean: 363.22,      Loss_min: 360.67, Loss_max: 368.19 
    

Epoch 47/4000, Loss_mean: 192.32,      Loss_min: 150.68, Loss_max: 257.11 
      Loss_mean_val: 206.38, Loss_min_val: 116.48,      Loss_max_val: 280.67 
      Loss_mean_trainval: 204.88
Epoch 48/4000, Loss_mean: 171.6,      Loss_min: 128.39, Loss_max: 211.91 
      Loss_mean_val: 206.21, Loss_min_val: 100.59,      Loss_max_val: 274.3 
      Loss_mean_trainval: 202.51
Epoch 49/4000, Loss_mean: 185.1,      Loss_min: 155.9, Loss_max: 211.88 
      Loss_mean_val: 207.59, Loss_min_val: 93.26,      Loss_max_val: 271.01 
      Loss_mean_trainval: 205.18
Epoch 50/4000, Loss_mean: 166.77,      Loss_min: 120.68, Loss_max: 215.82 
      Loss_mean_val: 204.98, Loss_min_val: 105.12,      Loss_max_val: 255.47 
      Loss_mean_trainval: 200.89
Epoch 51/4000, Loss_mean: 176.3,      Loss_min: 155.29, Loss_max: 201.02 
      Loss_mean_val: 201.42, Loss_min_val: 111.52,      Loss_max_val: 261.54 
      Loss_mean_trainval: 198.73
Epoch 52/4000, Loss_mean: 185.51,      Loss_min: 152.27, Loss_max: 253.75 
 

Epoch 92/4000, Loss_mean: 162.98,      Loss_min: 135.85, Loss_max: 215.05 
      Loss_mean_val: 185.3, Loss_min_val: 98.93,      Loss_max_val: 246.71 
      Loss_mean_trainval: 182.91
Epoch 93/4000, Loss_mean: 139.98,      Loss_min: 81.13, Loss_max: 203.03 
      Loss_mean_val: 182.71, Loss_min_val: 63.88,      Loss_max_val: 248.97 
      Loss_mean_trainval: 178.14
Epoch 94/4000, Loss_mean: 140.53,      Loss_min: 112.1, Loss_max: 177.67 
      Loss_mean_val: 183.77, Loss_min_val: 87.56,      Loss_max_val: 236.6 
      Loss_mean_trainval: 179.14
Epoch 95/4000, Loss_mean: 153.12,      Loss_min: 125.54, Loss_max: 186.22 
      Loss_mean_val: 181.2, Loss_min_val: 109.09,      Loss_max_val: 257.23 
      Loss_mean_trainval: 178.19
Epoch 96/4000, Loss_mean: 155.32,      Loss_min: 116.28, Loss_max: 207.19 
      Loss_mean_val: 183.17, Loss_min_val: 102.56,      Loss_max_val: 277.61 
      Loss_mean_trainval: 180.19
Epoch 97/4000, Loss_mean: 146.57,      Loss_min: 115.87, Loss_max: 182.87 
   

Epoch 137/4000, Loss_mean: 129.8,      Loss_min: 98.74, Loss_max: 155.0 
      Loss_mean_val: 173.22, Loss_min_val: 97.84,      Loss_max_val: 415.47 
      Loss_mean_trainval: 168.58
Epoch 138/4000, Loss_mean: 127.17,      Loss_min: 110.9, Loss_max: 159.27 
      Loss_mean_val: 170.67, Loss_min_val: 97.15,      Loss_max_val: 261.94 
      Loss_mean_trainval: 166.02
Epoch 139/4000, Loss_mean: 140.59,      Loss_min: 92.23, Loss_max: 193.95 
      Loss_mean_val: 168.44, Loss_min_val: 64.45,      Loss_max_val: 257.4 
      Loss_mean_trainval: 165.46
Epoch 140/4000, Loss_mean: 139.2,      Loss_min: 97.76, Loss_max: 197.32 
      Loss_mean_val: 168.24, Loss_min_val: 89.07,      Loss_max_val: 262.71 
      Loss_mean_trainval: 165.13
Epoch 141/4000, Loss_mean: 133.91,      Loss_min: 101.02, Loss_max: 167.95 
      Loss_mean_val: 169.37, Loss_min_val: 74.28,      Loss_max_val: 241.21 
      Loss_mean_trainval: 165.58
Epoch 142/4000, Loss_mean: 126.71,      Loss_min: 112.15, Loss_max: 138.22 
  

Epoch 182/4000, Loss_mean: 115.29,      Loss_min: 103.93, Loss_max: 140.6 
      Loss_mean_val: 156.24, Loss_min_val: 69.85,      Loss_max_val: 219.75 
      Loss_mean_trainval: 151.86
Epoch 183/4000, Loss_mean: 128.84,      Loss_min: 99.42, Loss_max: 195.56 
      Loss_mean_val: 159.21, Loss_min_val: 70.8,      Loss_max_val: 230.2 
      Loss_mean_trainval: 155.96
Epoch 184/4000, Loss_mean: 117.17,      Loss_min: 92.65, Loss_max: 138.27 
      Loss_mean_val: 157.53, Loss_min_val: 83.41,      Loss_max_val: 222.97 
      Loss_mean_trainval: 153.21
Epoch 185/4000, Loss_mean: 119.25,      Loss_min: 77.13, Loss_max: 147.59 
      Loss_mean_val: 155.8, Loss_min_val: 84.86,      Loss_max_val: 234.44 
      Loss_mean_trainval: 151.89
Epoch 186/4000, Loss_mean: 114.92,      Loss_min: 94.38, Loss_max: 132.32 
      Loss_mean_val: 155.08, Loss_min_val: 62.16,      Loss_max_val: 202.22 
      Loss_mean_trainval: 150.78
Epoch 187/4000, Loss_mean: 110.07,      Loss_min: 84.61, Loss_max: 136.1 
    

...

Epoch 3807/4000, Loss_mean: 5.07,      Loss_min: 3.81, Loss_max: 7.7 
      Loss_mean_val: 114.23, Loss_min_val: 40.07,      Loss_max_val: 237.6 
      Loss_mean_trainval: 102.54
Epoch 3808/4000, Loss_mean: 4.83,      Loss_min: 3.69, Loss_max: 6.04 
      Loss_mean_val: 111.01, Loss_min_val: 30.06,      Loss_max_val: 255.93 
      Loss_mean_trainval: 99.64
Epoch 3809/4000, Loss_mean: 5.1,      Loss_min: 4.1, Loss_max: 6.02 
      Loss_mean_val: 109.77, Loss_min_val: 28.14,      Loss_max_val: 247.97 
      Loss_mean_trainval: 98.56
Epoch 3810/4000, Loss_mean: 5.33,      Loss_min: 3.8, Loss_max: 6.86 
      Loss_mean_val: 119.52, Loss_min_val: 28.97,      Loss_max_val: 352.11 
      Loss_mean_trainval: 107.3
Epoch 3811/4000, Loss_mean: 5.67,      Loss_min: 3.67, Loss_max: 8.23 
      Loss_mean_val: 113.84, Loss_min_val: 48.11,      Loss_max_val: 278.82 
      Loss_mean_trainval: 102.26
Epoch 3812/4000, Loss_mean: 5.21,      Loss_min: 3.97, Loss_max: 6.42 
      Loss_mean_val: 114.13, Los

Epoch 3853/4000, Loss_mean: 5.6,      Loss_min: 4.08, Loss_max: 7.28 
      Loss_mean_val: 115.68, Loss_min_val: 50.85,      Loss_max_val: 241.53 
      Loss_mean_trainval: 103.89
Epoch 3854/4000, Loss_mean: 6.55,      Loss_min: 4.99, Loss_max: 9.45 
      Loss_mean_val: 117.82, Loss_min_val: 44.72,      Loss_max_val: 366.94 
      Loss_mean_trainval: 105.91
Epoch 3855/4000, Loss_mean: 8.47,      Loss_min: 5.53, Loss_max: 13.84 
      Loss_mean_val: 123.56, Loss_min_val: 35.3,      Loss_max_val: 257.85 
      Loss_mean_trainval: 111.23
Epoch 3856/4000, Loss_mean: 9.11,      Loss_min: 5.26, Loss_max: 11.99 
      Loss_mean_val: 125.55, Loss_min_val: 51.05,      Loss_max_val: 286.51 
      Loss_mean_trainval: 113.09
Epoch 3857/4000, Loss_mean: 11.31,      Loss_min: 7.61, Loss_max: 14.39 
      Loss_mean_val: 125.7, Loss_min_val: 36.17,      Loss_max_val: 339.44 
      Loss_mean_trainval: 113.45
Epoch 3858/4000, Loss_mean: 6.38,      Loss_min: 5.1, Loss_max: 9.19 
      Loss_mean_val: 117

Epoch 3899/4000, Loss_mean: 5.04,      Loss_min: 3.57, Loss_max: 6.96 
      Loss_mean_val: 115.63, Loss_min_val: 37.99,      Loss_max_val: 304.91 
      Loss_mean_trainval: 103.79
Epoch 3900/4000, Loss_mean: 4.49,      Loss_min: 3.62, Loss_max: 5.81 
      Loss_mean_val: 114.18, Loss_min_val: 49.71,      Loss_max_val: 261.3 
      Loss_mean_trainval: 102.44
Epoch 3901/4000, Loss_mean: 4.55,      Loss_min: 2.63, Loss_max: 6.39 
      Loss_mean_val: 115.15, Loss_min_val: 47.17,      Loss_max_val: 228.89 
      Loss_mean_trainval: 103.31
Epoch 3902/4000, Loss_mean: 4.72,      Loss_min: 3.12, Loss_max: 6.29 
      Loss_mean_val: 114.14, Loss_min_val: 37.57,      Loss_max_val: 265.54 
      Loss_mean_trainval: 102.42
Epoch 3903/4000, Loss_mean: 4.97,      Loss_min: 3.46, Loss_max: 7.39 
      Loss_mean_val: 108.21, Loss_min_val: 37.83,      Loss_max_val: 227.02 
      Loss_mean_trainval: 97.15
Epoch 3904/4000, Loss_mean: 4.39,      Loss_min: 2.99, Loss_max: 5.81 
      Loss_mean_val: 113.1

Epoch 3945/4000, Loss_mean: 4.43,      Loss_min: 4.0, Loss_max: 5.05 
      Loss_mean_val: 115.64, Loss_min_val: 44.41,      Loss_max_val: 234.18 
      Loss_mean_trainval: 103.73
Epoch 3946/4000, Loss_mean: 4.25,      Loss_min: 3.55, Loss_max: 5.37 
      Loss_mean_val: 111.49, Loss_min_val: 43.09,      Loss_max_val: 279.21 
      Loss_mean_trainval: 100.01
Epoch 3947/4000, Loss_mean: 4.51,      Loss_min: 3.19, Loss_max: 6.54 
      Loss_mean_val: 113.93, Loss_min_val: 44.78,      Loss_max_val: 343.53 
      Loss_mean_trainval: 102.21
Epoch 3948/4000, Loss_mean: 4.65,      Loss_min: 3.4, Loss_max: 6.6 
      Loss_mean_val: 115.16, Loss_min_val: 44.54,      Loss_max_val: 232.11 
      Loss_mean_trainval: 103.33
Epoch 3949/4000, Loss_mean: 4.14,      Loss_min: 1.94, Loss_max: 5.21 
      Loss_mean_val: 114.35, Loss_min_val: 49.31,      Loss_max_val: 282.17 
      Loss_mean_trainval: 102.55
Epoch 3950/4000, Loss_mean: 4.44,      Loss_min: 3.34, Loss_max: 5.39 
      Loss_mean_val: 114.2,

Epoch 3991/4000, Loss_mean: 4.75,      Loss_min: 2.8, Loss_max: 6.81 
      Loss_mean_val: 120.3, Loss_min_val: 38.72,      Loss_max_val: 304.77 
      Loss_mean_trainval: 107.93
Epoch 3992/4000, Loss_mean: 4.35,      Loss_min: 2.87, Loss_max: 5.84 
      Loss_mean_val: 115.19, Loss_min_val: 37.4,      Loss_max_val: 327.95 
      Loss_mean_trainval: 103.32
Epoch 3993/4000, Loss_mean: 4.69,      Loss_min: 2.76, Loss_max: 8.37 
      Loss_mean_val: 115.65, Loss_min_val: 41.17,      Loss_max_val: 262.6 
      Loss_mean_trainval: 103.77
Epoch 3994/4000, Loss_mean: 4.52,      Loss_min: 3.47, Loss_max: 5.31 
      Loss_mean_val: 117.77, Loss_min_val: 40.82,      Loss_max_val: 290.6 
      Loss_mean_trainval: 105.64
Epoch 3995/4000, Loss_mean: 4.52,      Loss_min: 3.5, Loss_max: 5.18 
      Loss_mean_val: 112.73, Loss_min_val: 36.11,      Loss_max_val: 295.03 
      Loss_mean_trainval: 101.14
Epoch 3996/4000, Loss_mean: 4.31,      Loss_min: 3.06, Loss_max: 5.67 
      Loss_mean_val: 111.46, L